
# Import neccessary modules
1 for matrix caculation, 2 for Deep Learning

In [1]:
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from module.input_parser_old import input_parser
from module import in_fun
from module.torch import BinaryClassifier


beacon_list = [f"beacon{idx + 1}" for idx in range(9)]

%matplotlib widget

/git/trajectory-correlation/module/in_fun.py:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(cur) is 0:
/git/trajectory-correlation/module/in_fun.py:30: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(cur) is 0:


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Load Data
python에서 csv파일을 읽어올때, 숫자만 있는것이 아닌 문자열이 포함된 데이터를 읽을 경우 느려지는 것 확인

In [3]:
data_path = "data/train/infra_free/"
room_list = [room + '/' for room in os.listdir(data_path)]
room_data_list = []
for room in room_list:
    file_list = [file for file in os.listdir(data_path + room) if file.endswith(".csv")]
    cur_room_file = []
    for file in file_list:
        cur_room_file.append(pd.read_csv(data_path + room + file, header=None).to_numpy())
    room_data_list.append(np.concatenate((cur_room_file), axis=0))

In [4]:
for room_data in room_data_list:
    print(np.shape(room_data))

(26, 101)
(22, 101)
(22, 101)
(22, 101)


# Data 만들기
같은 방 데이터 끼리 매칭시키고, 다른 방 데이터 끼리 매칭시키고

In [5]:
matched_same = []
for room_data in room_data_list:
    matched_same.append(in_fun.train_matcher(room_data[:,1:41], room_data[:,1:41]))
same_diff_ble = np.concatenate([same[0] for same in matched_same])
same_each_ble = np.concatenate([same[1] for same in matched_same])
print(np.shape(same_each_ble))

(2128, 40)


In [6]:
matched_diff = []
for idx in range(len(room_data_list)):
    if idx == 0:
        matched_diff.append(in_fun.train_matcher(room_data_list[-1][:,1:41], room_data_list[idx][:,1:41]))
    else:
        matched_diff.append(in_fun.train_matcher(room_data_list[idx-1][:,1:41], room_data_list[idx][:,1:41]))
    
diff_diff_ble = np.concatenate([diff[0] for diff in matched_diff])
diff_each_ble = np.concatenate([diff[1] for diff in matched_diff])
print(np.shape(diff_each_ble))

(2112, 40)


# Loading data for nn

In [7]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split

x_tensor_diff = torch.from_numpy(np.concatenate((same_diff_ble, diff_diff_ble))).float()
x_tensor_each = torch.from_numpy(np.concatenate((same_each_ble, diff_each_ble))).float()

y = np.concatenate((np.zeros([len(same_each_ble), 1]) + 1, np.zeros([len(diff_each_ble), 1])))
y_tensor = torch.from_numpy(y).float()

dataset_diff = TensorDataset(x_tensor_diff, y_tensor)
dataset_each = TensorDataset(x_tensor_each, y_tensor)

train_len_diff = len(dataset_diff) // 5 * 4
train_len_each = len(dataset_each) // 5 * 4
val_len_diff = len(dataset_diff) - train_len_diff
val_len_each  = len(dataset_each) - train_len_each

train_dataset_diff, val_dataset_diff = random_split(dataset_diff, [train_len_diff, val_len_diff])
train_dataset_each, val_dataset_each = random_split(dataset_each, [train_len_each, val_len_each])

train_loader_diff = DataLoader(dataset=train_dataset_diff, batch_size=32, shuffle=True)
train_loader_each = DataLoader(dataset=train_dataset_each, batch_size=32, shuffle=True)
val_loader_diff = DataLoader(dataset = val_dataset_diff, batch_size = val_len_diff)
val_loader_each = DataLoader(dataset = val_dataset_each, batch_size = val_len_each)

# Layer
nn 생성

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda:0


In [9]:
net_diff = BinaryClassifier(input_size = 20)
net_each = BinaryClassifier(input_size = 40)
net_diff.to(device)
net_each.to(device)
optimizer_diff = optim.SGD(net_diff.parameters(), lr=3e-3)
optimizer_each = optim.SGD(net_each.parameters(), lr=3e-3)
criterion = nn.BCEWithLogitsLoss()

비교분석을 위한 수치  
모든 데이터에 true라고 응답해도, 정확도가 62%는 나온다.

In [10]:
print(len(same_each_ble) / (len(same_each_ble) + len(diff_each_ble)))

0.5018867924528302


In [11]:
num_epoch = 100
for epoch in range(num_epoch):
    for batch_idx, data in enumerate(train_loader_diff):
        x_train, y_train = data[0].to(device), data[1].to(device)
        hypothesis = net_diff(x_train)
        cost = criterion(hypothesis, y_train)
        # cost로 H(x) 계산
        optimizer_diff.zero_grad()
        cost.backward()
        optimizer_diff.step()

    with torch.no_grad():
        for data in val_loader_diff:
            x_val, y_val = data[0].to(device), data[1].to(device)
            hypothesis = net_diff(x_val)
            prediction = hypothesis >= torch.FloatTensor([0.5]).to(device)
            correct_prediction = prediction.float() == y_val  # 실제값과 일치하는 경우만 True로 간주
            val_accuarcy = correct_prediction.sum().item() / len(correct_prediction)  # 정확도를 계산
            print(f"epoch: {epoch}, Val Accuarcy : {val_accuarcy * 100}%")


epoch: 0, Val Accuarcy : 49.410377358490564%
epoch: 1, Val Accuarcy : 49.410377358490564%
epoch: 2, Val Accuarcy : 49.410377358490564%
epoch: 3, Val Accuarcy : 49.410377358490564%
epoch: 4, Val Accuarcy : 49.410377358490564%
epoch: 5, Val Accuarcy : 49.410377358490564%
epoch: 6, Val Accuarcy : 49.410377358490564%
epoch: 7, Val Accuarcy : 49.410377358490564%
epoch: 8, Val Accuarcy : 49.410377358490564%
epoch: 9, Val Accuarcy : 49.410377358490564%
epoch: 10, Val Accuarcy : 49.410377358490564%
epoch: 11, Val Accuarcy : 50.0%
epoch: 12, Val Accuarcy : 53.18396226415094%
epoch: 13, Val Accuarcy : 60.37735849056604%
epoch: 14, Val Accuarcy : 66.50943396226415%
epoch: 15, Val Accuarcy : 72.75943396226415%
epoch: 16, Val Accuarcy : 77.7122641509434%
epoch: 17, Val Accuarcy : 80.54245283018868%
epoch: 18, Val Accuarcy : 83.9622641509434%
epoch: 19, Val Accuarcy : 87.02830188679245%
epoch: 20, Val Accuarcy : 88.56132075471697%
epoch: 21, Val Accuarcy : 90.33018867924528%
epoch: 22, Val Accuarcy 

In [12]:
num_epoch = 100
for epoch in range(num_epoch):
    for batch_idx, data in enumerate(train_loader_each):
        x_train, y_train = data[0].to(device), data[1].to(device)
        hypothesis = net_each(x_train)
        cost = criterion(hypothesis, y_train)
        # cost로 H(x) 계산
        optimizer_each.zero_grad()
        cost.backward()
        optimizer_each.step()

    with torch.no_grad():
        for data in val_loader_each:
            x_val, y_val = data[0].to(device), data[1].to(device)
            hypothesis = net_each(x_val)
            prediction = hypothesis >= torch.FloatTensor([0.5]).to(device)
            correct_prediction = prediction.float() == y_val  # 실제값과 일치하는 경우만 True로 간주
            val_accuarcy = correct_prediction.sum().item() / len(correct_prediction)  # 정확도를 계산
            print(f"epoch: {epoch}, Val Accuarcy : {val_accuarcy * 100}%")


epoch: 0, Val Accuarcy : 53.18396226415094%
epoch: 1, Val Accuarcy : 53.18396226415094%
epoch: 2, Val Accuarcy : 53.18396226415094%
epoch: 3, Val Accuarcy : 53.18396226415094%
epoch: 4, Val Accuarcy : 53.18396226415094%
epoch: 5, Val Accuarcy : 53.18396226415094%
epoch: 6, Val Accuarcy : 53.18396226415094%
epoch: 7, Val Accuarcy : 53.18396226415094%
epoch: 8, Val Accuarcy : 53.18396226415094%
epoch: 9, Val Accuarcy : 53.18396226415094%
epoch: 10, Val Accuarcy : 53.18396226415094%
epoch: 11, Val Accuarcy : 53.18396226415094%
epoch: 12, Val Accuarcy : 53.655660377358494%
epoch: 13, Val Accuarcy : 56.95754716981132%
epoch: 14, Val Accuarcy : 62.264150943396224%
epoch: 15, Val Accuarcy : 67.80660377358491%
epoch: 16, Val Accuarcy : 74.41037735849056%
epoch: 17, Val Accuarcy : 79.95283018867924%
epoch: 18, Val Accuarcy : 83.37264150943396%
epoch: 19, Val Accuarcy : 86.43867924528303%
epoch: 20, Val Accuarcy : 88.91509433962264%
epoch: 21, Val Accuarcy : 90.2122641509434%
epoch: 22, Val Accu

# 개선 계획
학습 과정에서 accuracy가 높아지지 않은 것으로, 제대로 학습되지 않는 상태인것으로 확인  
input에 대한 조절로 결과를 얻고자한다
1. RSSI 값 연산 시 재정의:   
현재 해당 비컨에 해당하는 RSSI가 없으면, 0으로 넣어주고 비교하고있다.  
그결과, null데이터와 -90dB의 차이가, null데이터와 -10dB의 차이보다 더 큰 상태이다.  
null값에 대해 -100을 넣어주고 연산하는 방식으로 오류를 줄여야 한다.  
2. Input 범위 재설정:  
1의 적용 이후, 실제 input으로 들어갈 값들은 -100 ~ -1 사이의 값이다.  
input에 100을 더한 후 100으로 나눠주어, input들이 0 ~ 1 사이의 범위에 있도록 normalize 해줄 계획이다
3. Input Data 추가:  
현재는 두 데이터간의 '차의 절댓값' 만을 넣어주고 있어서, 데이터 dimension이 9이하인 상태이다. (적은 편)  
차만을 사용하는 과정에서, 각 데이터의 RF measurement들이 버려지게 된다.  
4. Input의 다원화:  
현재 사용하고 있지 않은 기압값등을 어떻게 활용할지 고려한다.

In [13]:
torch.save(net_diff.state_dict(), "./model/diff")
torch.save(net_each.state_dict(), "./model/each")